In [2]:
import geopandas as gpd
import pandas as pd 
import json
from area import area

file_path_of_forests = 'source/waldflaechen.geojson'
forests = gpd.read_file(file_path_of_forests)

forests.insert(0, 'New_ID', range(0, 0 + len(forests)))
forests.to_file("output_waldflaechen.geojson", driver="GeoJSON")

forests["area_text"] = (forests["area_text"].replace(',', '.', regex=True).astype(float))

isBerlinForest = forests['land'] == 'Berlin'
berlinForests = forests[isBerlinForest]
# print(len(forests)) # 11522
# print(len(berlinForests)) # 6258

# berlinForests['area_text'].sum() # 282036587.22155315
# berlinForests.to_file('berlinForests.geojson', driver='GeoJSON')

berlinForest = berlinForests.unary_union
berlinForest.crs = "EPSG:5678"
frame = gpd.GeoSeries([berlinForest])
test = gpd.GeoSeries([berlinForest]).to_json()

y = json.loads(test)
y = y['features'][0]['geometry']
waldflaeche = area(y) 
# print(waldflaeche) # 162857690.6703506 in m^2

11522


In [3]:
file_path_of_open_spaces = 'gruenflaechen.geojson'
open_spaces = gpd.read_file(file_path_of_open_spaces)

gruenflaeche = open_spaces['KATASTERFL'].sum()
# print(gruenflaeche) # 61475596.6
# print(len(open_spaces)) # 2525
gesamt = waldflaeche + gruenflaeche
print(gesamt/10000)
# 22433.32872703506

22433.32872703506
